In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
df = pd.read_csv('/kaggle/input/rbalogin-dataset/data/preprocessed.csv')

In [ ]:
tqdm.pandas()

# Dataset Introduction
- 출처 : https://dl.acm.org/doi/abs/10.1145/3546069
- 2020년 2월부터 2021년 2월까지 노르웨이 통신사 Telenor에 속한 SSO 서비스에서 로그인한 330만 명의 사용자의 3,300만 로그인 기록 데이터를 바탕으로 합성된 데이터셋
- 대륙별 로그인 데이터

|대륙명|Data size|소속 국가 개수|
|:---:|:---:|:---:|
|Europe |18,912,729|55|
|Americas|10,238,934|54|
|Asia|1,525,063|50|
|Oceania|436,658|48|
|Africa|155,801|20|


- 학습 가능한 데이터 사이즈이면서 국가별 다양한 로그인 기록을 가지고 있는 Asia 대륙에 한해 데이터를 Select하여 프로젝트 진행

In [ ]:
df.head()

In [ ]:
df.columns

# Dataset Description
## Target Variable
- is attack ip, is takeover 데이터를 결합하여 Target 변수(label)로 지정
- 전체 로그인 공격 비율 약 8%

In [ ]:
df['label'].mean()

## Independent Variable
- 데이터프레임 내 존재하는 'user id', 'country_code', 'region', 'city', 'name', 'browser name and version', 'os name and version', 'device type', 'rtt', 'timestamp', 'login_success'의 독립변수와 타겟 레이블 사이 연관성 분석
- 주어진 데이터 항목을 바탕으로 'country', 'region risk grade', 'city risk grade', 'name risk grade', 'browser name', 'browser is legacy', 'os name', 'os is legacy'의 독립변수를 새로 생성하고, 이들과 타겟 레이블 사이의 연관성 추가 분석

### 01. User ID

- 해당 변수는 공격자가 타겟팅하고자 하는 대상에 대한 데이터이므로 공격 여부에 영향을 주지 않는 Nuisance Variable이라고 규정, EDA 분석에서 제외하였음.

### 02. Country
- country별 로그인 기록 횟수, 공격 비율

#### 02-1. Country별 로그인 기록 횟수, 공격 비율

In [ ]:
# temp = data.groupby('Country').agg({'Is Threat': 'mean','sub-region': 'size'})
# temp.rename(columns={'sub-region': 'size'}, inplace=True)
# temp = temp.reset_index()
# temp = temp.sort_values(by = 'size',ascending = False)
# fig, ax1 = plt.subplots(figsize=(14, 8))


# sns.barplot(x='Country', y='size', data=temp, ax=ax1, color='lightblue')
# ax1.set_ylabel('Count', color='blue')
# ax1.tick_params(axis='y', colors='blue')
# ax1.set_xticklabels(ax1.get_xticklabels(), rotation=45, ha="right")


# ax2 = ax1.twinx()
# sns.lineplot(x='Country', y='Is Threat', data=temp, ax=ax2, color='darkred', marker='o')
# ax2.set_ylabel('Is Threat Proportion', color='darkred')
# ax2.tick_params(axis='y', colors='darkred')

# plt.title('Count and Is Threat Proportion by Country')
# plt.show()

In [ ]:
login_df = df.groupby('country').count().sort_values(by = 'label', ascending = False).reset_index(drop = False)
fig = sns.barplot(login_df[:20], x = 'country', y = 'label')
for item in fig.get_xticklabels():
    item.set_rotation(75)
    item.set_ha('right')
frequent_countries = login_df['country'][:20]

In [ ]:
ratio_df = df.groupby('country').mean(numeric_only = True).sort_values(by = 'label', ascending = False).reset_index(drop = False)
fig = sns.barplot(ratio_df[:20], x = 'country', y = 'label')
for item in fig.get_xticklabels():
    item.set_rotation(75)
    item.set_ha('right')

In [ ]:
merged_df = pd.merge(left = login_df, right = ratio_df, how = 'left', on = 'country')
merged_df['label'] = merged_df.progress_apply(lambda x : x['label_x'] * x['label_y'], axis = 1).astype(int)
total_df = merged_df[['country', 'label_x']].copy()
total_df.columns = ['country', 'label']
total_df['type'] = 'Total'
attack_df = merged_df[['country', 'label']].copy()
attack_df['type'] = 'Attack'
merged_df = pd.concat([total_df, attack_df])

In [ ]:
fig = sns.barplot(merged_df[merged_df['country'].isin(frequent_countries.values)], x = 'country', y = 'label', hue = 'type')
for item in fig.get_xticklabels():
    item.set_rotation(45)
    item.set_ha('right')

- 가장 많은 로그인 횟수를 보인 나라는 인도
- 인도네시아의 경우 두번째로 가장 많은 로그인 회수를 보이면서, 공격비율도 매우 높게 기록됨
- 가장 높은 공격 비율을 기록한 나라는 베트남
- 로그인 공격에 있어 나라 정보는 중요한 Feature로 최대한 가공하지 않고 모델에 Input할 수 있는 방법 고안

### 03. Region

In [ ]:
region_df = df.groupby('region').count().sort_values(by = 'label', ascending = False).reset_index(drop = False)

In [ ]:
region_df

- 모델에 Input하기에는 범주가 커 차원이 너무 커질 수 있는 문제점 존재
- 공격 비율에 따라 Risk grade를 4단계로 측정하여 Input하는 방법 고안

#### 03-1. Region별 분석
- 데이터가 존재하는 범주 내에서 각 지역 데이터에 따른 분석

In [ ]:
fig = sns.barplot(region_df[:20], x = 'region', y = 'label')
for item in fig.get_xticklabels():
    item.set_rotation(45)
    item.set_ha('right')
frequent_regions = region_df['region'][1:21]

In [ ]:
fig = sns.barplot(region_df[1:21], x = 'region', y = 'label')
for item in fig.get_xticklabels():
    item.set_rotation(45)
    item.set_ha('right')

- 공격자의 지역 변수가 알려지지 않은 경우가 가장 많음.
- 그 외의 경우, 마하라슈트라쥬(인도), 서부 벵골 (방글라데시), 구자라트 (인도), 발리 (인도네시아), 다카주 (방글라데시) 등 에서의 순으로 접속이 많이 발생하며, 주로 동남아시아 또는 남부 아시아 지역에서의 접근이 다수 발생.

In [ ]:
ratio_df = df.groupby('region').mean(numeric_only = True).sort_values(by = 'label', ascending = False).reset_index(drop = False)
fig = sns.barplot(ratio_df[:20], x = 'region', y = 'label')
for item in fig.get_xticklabels():
    item.set_rotation(45)
    item.set_ha('right')
region_in_focus = ratio_df['region'][:20]

In [ ]:
fig = sns.barplot(region_df[region_df['region'].isin(region_in_focus.values)], x = 'region', y = 'label')
for item in fig.get_xticklabels():
    item.set_rotation(45)
    item.set_ha('right')
sns.lineplot(x = np.arange(-1, 21, 1), y = 5)

- 비율적 측면에서는 많은 지역에서 100%의 공격률을 보이고 있으나, 해당 지역들에서의 접속 횟수가 유의미하지 않은 경우 (예: 5회 미만 접속 사례) 가 많이 발생하고 있음.

In [ ]:
ratio_df = df.groupby('region').mean(numeric_only = True).sort_values(by = 'label', ascending = False).reset_index(drop = False)
fig = sns.barplot(ratio_df[36:56], x = 'region', y = 'label')
for item in fig.get_xticklabels():
    item.set_rotation(45)
    item.set_ha('right')
region_in_focus = ratio_df['region'][36:56]

In [ ]:
fig = sns.barplot(region_df[region_df['region'].isin(region_in_focus.values)], x = 'region', y = 'label')
for item in fig.get_xticklabels():
    item.set_rotation(45)
    item.set_ha('right')
sns.lineplot(x = np.arange(-1, 21, 1), y = 5)

- 공격 성공률이 100%인 지역을 제외하면, 보다 유의미한 접속 횟수를 가지고 있는 지역의 비율이 더 높아지는 경향이 존재함.

In [ ]:
merged_df = pd.merge(left = region_df, right = ratio_df, how = 'left', on = 'region')
merged_df['label'] = merged_df.progress_apply(lambda x : x['label_x'] * x['label_y'], axis = 1).astype(int)
total_df = merged_df[['region', 'label_x']].copy()
total_df.columns = ['region', 'label']
total_df['type'] = 'Total'
attack_df = merged_df[['region', 'label']].copy()
attack_df['type'] = 'Attack'
merged_df = pd.concat([total_df, attack_df])

In [ ]:
fig = sns.barplot(merged_df[merged_df['region'].isin(frequent_regions.values)], x = 'region', y = 'label', hue = 'type')
for item in fig.get_xticklabels():
    item.set_rotation(75)
    item.set_ha('right')

- 접속 횟수와 접속 비율을 모두 고려하였을 때, 발리 (동남아시아), 구자라트 (인도), 서부 자바 (인도네이사), 동부 자바 (인도네시아), 자카르타 (인도네이사) 등에서의 공격률이 두드러짐.
- 주로 동남아시아 또는 인도 계열 지역에서의 공격 사례가 많이 발견됨.

#### 03-2. Region Risk Grade에 따른 분석
- 각 지역별 공격률에 따라서 지역당 위험도를 4개의 스케일로 분류
- 각 지역별 위험도에 따른 접속, 공격 횟수 및 비율 분석

In [ ]:
region_risk_df = df.groupby('region_risk_grade').count().reset_index(drop = False)

In [ ]:
region_risk_df['label']

In [ ]:
sns.barplot(region_risk_df, x = 'region_risk_grade', y = 'label')

- 각 지역별 위험도를 측정하였을 때, 거의 대부분이 지역이 위험도 1 (0.2~0.4 사이의 비율)을 가지고 있는 것으로 파악됨. 

In [ ]:
sns.barplot(df, x = 'region_risk_grade', y = 'label')

- 각 위험도별 공격 비율은 위의 그림과 같음.

### 04. City

In [ ]:
city_df = df.groupby('city').count().sort_values('label', ascending = False).reset_index(drop = False)

In [ ]:
city_df

- 가장 낮은 수준에서, 각 도시별 공격 횟수 및 공격 비율 분석
- Region 독립 변수와 마찬가지로, 모델에 Input하기에는 범주가 커 차원이 너무 커질 수 있는 문제점 존재
- 공격 비율에 따라 Risk grade를 4단계로 측정하여 Input하는 방법 고안

#### 04-1. City Name

In [ ]:
fig = sns.barplot(city_df[:20], x = 'city', y = 'label')
for item in fig.get_xticklabels():
    item.set_rotation(75)
    item.set_ha('right')

In [ ]:
fig = sns.barplot(city_df[1:21], x = 'city', y = 'label')
for item in fig.get_xticklabels():
    item.set_rotation(75)
    item.set_ha('right')
city_in_focus = city_df['city'][1:21]

- Region의 경우와 마찬가지로, 사용자의 도시 정보에 대한 데이터는 주어지지 않은 경우가 대부분.
- 도시정보가 주어진 경우는 콜카타 (인도), 덴파사르 (인도네시아), 뭄바이 (인도) 등에서의 접속 횟수가 가장 많음.

In [ ]:
ratio_df = df.groupby('city').mean(numeric_only = True).sort_values(by = 'label', ascending = False).reset_index(drop = False)
fig = sns.barplot(ratio_df[:20], x = 'city', y = 'label')
for item in fig.get_xticklabels():
    item.set_rotation(45)
    item.set_ha('right')
city_in_focus_in_ratio = ratio_df['city'][:20]

In [ ]:
fig = sns.barplot(city_df[city_df['city'].isin(city_in_focus_in_ratio.values)], x = 'city', y = 'label')
for item in fig.get_xticklabels():
    item.set_rotation(45)
    item.set_ha('right')
sns.lineplot(x = np.arange(-1, 21, 1), y = 5)

- 100%에 가까운 공격 성공률을 가지고 있는 도시들의 경우, 접속 횟수 자체가 극단적으로 적게 나타나는 경향을 보임.

In [ ]:
merged_df = pd.merge(left = city_df, right = ratio_df, how = 'left', on = 'city')
merged_df['label'] = merged_df.progress_apply(lambda x : x['label_x'] * x['label_y'], axis = 1).astype(int)
total_df = merged_df[['city', 'label_x']].copy()
total_df.columns = ['city', 'label']
total_df['type'] = 'Total'
attack_df = merged_df[['city', 'label']].copy()
attack_df['type'] = 'Attack'
merged_df = pd.concat([total_df, attack_df])

In [ ]:
fig = sns.barplot(merged_df[merged_df['city'].isin(frequent_regions.values)], x = 'city', y = 'label', hue = 'type')
for item in fig.get_xticklabels():
    item.set_rotation(75)
    item.set_ha('right')

- 주요 도시별 공격률을 비교한 결과, 자라크타 (인도네시아) 등에서의 공격률이 두드러짐.

#### 04-2. City Risk Grade에 따른 분석
- 각 도시별 공격률에 따라서 지역당 위험도를 4개의 스케일로 분류
- 각 도시별 위험도에 따른 접속, 공격 횟수 및 비율 분석

In [ ]:
city_risk_df = df.groupby('city_risk_grade').count().reset_index(drop = False)

In [ ]:
city_risk_df['label']

In [ ]:
sns.barplot(city_risk_df, x = 'city_risk_grade', y = 'label')

- 각 지역별 위험도를 측정하였을 때, 거의 대부분이 지역이 위험도 1 (0.2 \~ 0.4 사이의 비율)을 가지고 있는 것으로 파악되며, 위험도가 0인 지역 (0.0 \~ 0.2 사이의 비율) 이 그 다음으로 많음.

In [ ]:
sns.barplot(df, x = 'city_risk_grade', y = 'label')

- 각 위험도별 공격 비율은 위의 그림과 같음.

### 05. ARN (Name)
- IP 주소를 서비스하고 있는 도메인당 할당된 서비스 제공 업체에 따른 분류

#### 05-1. ARN Name
- 데이터 내 주어져 있는 ARN 번호를 외부 데이터를 이용하여 실제 기업명과의 매핑을 진행.

In [ ]:
arn_df = df.groupby('name').count().sort_values('label', ascending = False).reset_index(drop = False)

In [ ]:
arn_df['name'] = arn_df['name'].progress_apply(lambda x : x.lower())

In [ ]:
arn_df

In [ ]:
fig = sns.barplot(arn_df[:20], x = 'name', y = 'label')
for item in fig.get_xticklabels():
    item.set_rotation(75)
    item.set_ha('right')
arn_in_focus = arn_df['name'][:20]

- 상당한 비율의 데이터에 대하여 ARN의 정보가 주어진 경우가 많음.
- ARN 제공 기업의 이름은 다음과 같은 규칙을 따르고 있음.
    1. 

In [ ]:
ratio_df = df.groupby('name').mean(numeric_only = True).sort_values(by = 'label', ascending = False).reset_index(drop = False)
fig = sns.barplot(ratio_df[:20], x = 'name', y = 'label')
for item in fig.get_xticklabels():
    item.set_rotation(45)
    item.set_ha('right')
arn_in_focus_in_ratio = ratio_df['name'][:20].progress_apply(lambda x : x.lower())

In [ ]:
fig = sns.barplot(arn_df[arn_df['name'].isin(arn_in_focus_in_ratio.values)], x = 'name', y = 'label')
for item in fig.get_xticklabels():
    item.set_rotation(45)
    item.set_ha('right')
sns.lineplot(x = np.arange(-1, 21, 1), y = 5)

- 100%에 가까운 공격 성공률을 가지고 있는 ARN의 경우, 타 feature에 비하여 유의미한 특징을 가지고 있는 유의미한 ARN을 찾을 수 있음.

In [ ]:
ratio_df['name'] = ratio_df['name'].progress_apply(lambda x : x.lower())
merged_df = pd.merge(left = arn_df, right = ratio_df, how = 'left', on = 'name')
merged_df['label'] = merged_df.progress_apply(lambda x : x['label_x'] * x['label_y'], axis = 1).astype(int)
total_df = merged_df[['name', 'label_x']].copy()
total_df.columns = ['name', 'label']
total_df['type'] = 'Total'
attack_df = merged_df[['name', 'label']].copy()
attack_df['type'] = 'Attack'
merged_df = pd.concat([total_df, attack_df])

In [ ]:
fig = sns.barplot(merged_df[merged_df['name'].isin(arn_in_focus.values)], x = 'name', y = 'label', hue = 'type')
for item in fig.get_xticklabels():
    item.set_rotation(75)
    item.set_ha('right')

- 주요 ARN별 공격률을 기록한 결과, ARN이 공개되지 않았거나, ARN 이름 중 특정 단어 (cheap 등)가 포함된 경우, ARN 이름 내 고유명사를 제외한 외국어가 포함된 경우 (cizgi telekomunikasyon anonim sirketi) 공격률이 두드러짐.

#### 05-2. ARN Risk Grade
- 각 ARN별 공격률에 따라서 지역당 위험도를 4개의 스케일로 분류
- 각 ARN별 위험도에 따른 접속, 공격 횟수 및 비율 분석

In [ ]:
arn_risk_df = df.groupby('name_risk_grade').count().reset_index(drop = False)

In [ ]:
arn_risk_df['label']

In [ ]:
sns.barplot(arn_risk_df, x = 'name_risk_grade', y = 'label')

- 각 지역별 위험도를 측정하였을 때, 거의 대부분의 ARN이 위험도 1 (0.2 \~ 0.4 사이의 비율)을 가지고 있는 것으로 파악되며, 위험도가 0인 ARN (0.0 \~ 0.2 사이의 비율), 위험도가 2인 ARN (0.4 \~ 0.6) 이 그 다음으로 많음.

In [ ]:
sns.barplot(df, x = 'name_risk_grade', y = 'label')

- 각 위험도별 공격 비율은 위의 그림과 같음.

### 06. Browser (Name & is Legacy)
- 사용자가 접속하고 있는 브라우저의 종류에 따른 분석.

#### 06-1. Browser Name

In [ ]:
browser_df = df.groupby('browser_name').count().sort_values('label', ascending = False).reset_index(drop = False)

In [ ]:
browser_df

In [ ]:
fig = sns.barplot(browser_df[:20], x = 'browser_name', y = 'label')
for item in fig.get_xticklabels():
    item.set_rotation(75)
    item.set_ha('right')
browser_in_focus = browser_df[:20]['browser_name']

- 사용자들은 대부분 크롬 기반 브라우저 또는 파이어폭스, 안드로이드 등에 기반한 브라우저를 사용.
- 브라우저의 이름 중에는 'Bot'이라는 단어가 포함되어 있는 경우도 있음.
- 사람들이 주로 사용하는 브라우저 외에도 다양한 브라우저가 있기에, 해당 브라우저 소속 여부를 추가 feature로 사용.

In [ ]:
ratio_df = df.groupby('browser_name').mean(numeric_only = True).sort_values(by = 'label', ascending = False).reset_index(drop = False)
fig = sns.barplot(ratio_df[:20], x = 'browser_name', y = 'label')
for item in fig.get_xticklabels():
    item.set_rotation(45)
    item.set_ha('right')
browser_in_focus_in_ratio = ratio_df['browser_name'][:20].progress_apply(lambda x : x.lower())

- 높은 공격률을 보이고 있는 브라우저들은 몇몇 특징을 보임
- 브라우저 이름 내 'Bot', 'Crawl' 등의 단어를 포함함.
- Chrome, Firefox 등 주로 사용하는 브라우저가 포함되어 있지 않음.

In [ ]:
merged_df = pd.merge(left = browser_df, right = ratio_df, how = 'left', on = 'browser_name')
merged_df['label'] = merged_df.progress_apply(lambda x : x['label_x'] * x['label_y'], axis = 1).astype(int)
total_df = merged_df[['browser_name', 'label_x']].copy()
total_df.columns = ['browser_name', 'label']
total_df['type'] = 'Total'
attack_df = merged_df[['browser_name', 'label']].copy()
attack_df['type'] = 'Attack'
merged_df = pd.concat([total_df, attack_df])

In [ ]:
fig = sns.barplot(merged_df[merged_df['browser_name'].isin(browser_in_focus.values)], x = 'browser_name', y = 'label', hue = 'type')
for item in fig.get_xticklabels():
    item.set_rotation(75)
    item.set_ha('right')

- 가장 많이 이용하는 브라우저인 크롬에서의 공격률이 적지 않음.
- 레거시 브라우저 여부 외에도 브라우저 이름 내 유명 기업명이 포함되는지 여부도 유효한 feature로 사용될 수 있음.

#### 06-2. Browser is Legacy

In [ ]:
fig = sns.barplot(df, x = 'browser_is_legacy', y = 'label')

- 브라우저가 사용자들이 주로 사용하고 있는 브라우저가 아닐 경우, 주로 사용하는 브라우저일 경우에 비하여 비하여 높은 공격률을 보임.

### 07. OS (Name & is Legacy)

#### 07-1. OS Name

In [ ]:
os_df = df.groupby('os_name').count().sort_values('label', ascending = False).reset_index(drop = False)

In [ ]:
os_df

In [ ]:
fig = sns.barplot(os_df[:20], x = 'os_name', y = 'label')
for item in fig.get_xticklabels():
    item.set_rotation(75)
    item.set_ha('right')
os_in_focus = os_df[:20]['os_name']

- 사용자들은 대부분 iOS, Mac OS X, Android를 사용.
- Other에 해당하는 운영체제를 사용하는 경우가 많음.
- 사람들이 주로 사용하는 OS 외에도 다양한 운영체제가 있기에, 해당 OS 소속 여부를 추가 feature로 사용.

In [ ]:
ratio_df = df.groupby('os_name').mean(numeric_only = True).sort_values(by = 'label', ascending = False).reset_index(drop = False)
fig = sns.barplot(ratio_df[:20], x = 'os_name', y = 'label')
for item in fig.get_xticklabels():
    item.set_rotation(45)
    item.set_ha('right')
os_in_focus_in_ratio = ratio_df['os_name'][:20]

- 높은 공격률을 보이고 있는 운영체제에는 Mac OS X 등 주로 사용하는 운영체제의 비율이 적음.

In [ ]:
merged_df = pd.merge(left = os_df, right = ratio_df, how = 'left', on = 'os_name')
merged_df['label'] = merged_df.progress_apply(lambda x : x['label_x'] * x['label_y'], axis = 1).astype(int)
total_df = merged_df[['os_name', 'label_x']].copy()
total_df.columns = ['os_name', 'label']
total_df['type'] = 'Total'
attack_df = merged_df[['os_name', 'label']].copy()
attack_df['type'] = 'Attack'
merged_df = pd.concat([total_df, attack_df])

In [ ]:
fig = sns.barplot(merged_df[merged_df['os_name'].isin(os_in_focus.values)], x = 'os_name', y = 'label', hue = 'type')
for item in fig.get_xticklabels():
    item.set_rotation(75)
    item.set_ha('right')

In [ ]:
fig = sns.barplot(merged_df[merged_df['os_name'].isin(os_in_focus_in_ratio.values)], x = 'os_name', y = 'label', hue = 'type')
for item in fig.get_xticklabels():
    item.set_rotation(75)
    item.set_ha('right')

#### 07-2. OS is Legacy

In [ ]:
fig = sns.barplot(df, x = 'os_is_legacy', y = 'label')

- 사용자가 사람들이 주로 사용하지 않는 운영체제를 사용할 경우 공격률이 높았음.

### 08. Device Type

In [ ]:
device_df = df.groupby('device_type').count().sort_values('label', ascending = False).reset_index(drop = False)

In [ ]:
sns.barplot(device_df, x = 'device_type', y = 'label')

- 사람들은 주로 모바일이나 데스크탑 환경에서 접속을 선호하며, 접속 환경이 주어지지 않은 경우도 존재.

In [ ]:
sns.barplot(df, x = 'device_type', y = 'label')

- 사용자가 Bot으로 분류될 경우 공격률이 유의미하게 높음.